In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open("vocab.distances.json", "r") as f:
    vocab_distances = json.load(f)

In [3]:
heuristic = []
for row in vocab_distances:
    current_word = {
        "word_name": row["word_name"],
        "heuristic": 0.0}
    for n in row["neighbors"][1:]:
        current_word["heuristic"] += 1.0 / n["word_distance"]
    heuristic.append(current_word)
    row["heuristic"] = current_word["heuristic"]

In [4]:
for row in vocab_distances[20:25]:
    print("Word: \"%s\" Heuristic: %.3f" % (row["word_name"], row["heuristic"]))
    for n in row["neighbors"][1:]:
        print("    Neighbor: \"%s\" Distance: %.3f" % (n["word_name"], n["word_distance"]))

Word: "are" Heuristic: 5.010
    Neighbor: "is" Distance: 3.122
    Neighbor: "was" Distance: 3.394
    Neighbor: "were" Distance: 3.501
    Neighbor: "be" Distance: 3.733
    Neighbor: "thats" Distance: 3.796
    Neighbor: "completely" Distance: 3.831
    Neighbor: "'re" Distance: 3.869
    Neighbor: "clearly" Distance: 3.875
    Neighbor: "neatly" Distance: 3.888
    Neighbor: "slightly" Distance: 3.896
    Neighbor: "nearly" Distance: 3.909
    Neighbor: "almost" Distance: 3.916
    Neighbor: "currently" Distance: 3.926
    Neighbor: "ladies" Distance: 3.932
    Neighbor: "others" Distance: 3.934
    Neighbor: "nicely" Distance: 3.941
    Neighbor: "happily" Distance: 3.949
    Neighbor: "seemingly" Distance: 3.956
    Neighbor: "jetliner" Distance: 3.958
Word: "next" Heuristic: 4.686
    Neighbor: "adjacent" Distance: 3.656
    Neighbor: "attached" Distance: 3.857
    Neighbor: "connected" Distance: 3.915
    Neighbor: "affixed" Distance: 3.965
    Neighbor: "tending" Distance: 3.9

In [5]:
heuristic = sorted(heuristic, key=(lambda x: -x["heuristic"]))
for row in heuristic:
    print("Word: \"%s\" Heuristic: %.3f" % (row["word_name"], row["heuristic"]))

Word: "<S>" Heuristic: 18.342
Word: "eatables" Heuristic: 14.551
Word: "outlets" Heuristic: 14.459
Word: "smoker" Heuristic: 14.458
Word: "<UNK>" Heuristic: 14.430
Word: "battling" Heuristic: 14.429
Word: "cleaners" Heuristic: 14.422
Word: "toucan" Heuristic: 14.418
Word: "marathon" Heuristic: 14.386
Word: "rubbish" Heuristic: 14.384
Word: "hippopotamus" Heuristic: 14.377
Word: "<UNK>" Heuristic: 14.362
Word: "<UNK>" Heuristic: 14.361
Word: "carcass" Heuristic: 14.360
Word: "installations" Heuristic: 14.359
Word: "<UNK>" Heuristic: 14.354
Word: "railways" Heuristic: 14.353
Word: "susan" Heuristic: 14.350
Word: "cross-legged" Heuristic: 14.350
Word: "potatos" Heuristic: 14.348
Word: "crayons" Heuristic: 14.346
Word: "<UNK>" Heuristic: 14.340
Word: "undone" Heuristic: 14.332
Word: "duster" Heuristic: 14.332
Word: "boast" Heuristic: 14.329
Word: "<UNK>" Heuristic: 14.328
Word: "<UNK>" Heuristic: 14.325
Word: "tarmacs" Heuristic: 14.324
Word: "slideshow" Heuristic: 14.314
Word: "flowerpot"

Word: "junction" Heuristic: 13.925
Word: "tun" Heuristic: 13.925
Word: "gril" Heuristic: 13.925
Word: "sights" Heuristic: 13.925
Word: "<UNK>" Heuristic: 13.925
Word: "foilage" Heuristic: 13.925
Word: "palms" Heuristic: 13.925
Word: "bodacious" Heuristic: 13.925
Word: "sparklers" Heuristic: 13.925
Word: "pandas" Heuristic: 13.924
Word: "siding" Heuristic: 13.924
Word: "spaceship" Heuristic: 13.924
Word: "boeing" Heuristic: 13.924
Word: "]" Heuristic: 13.924
Word: "chariot" Heuristic: 13.924
Word: "<UNK>" Heuristic: 13.924
Word: "tweed" Heuristic: 13.924
Word: "newscast" Heuristic: 13.923
Word: "otters" Heuristic: 13.923
Word: "purposes" Heuristic: 13.923
Word: "cassette" Heuristic: 13.923
Word: "vegas" Heuristic: 13.923
Word: "tick" Heuristic: 13.923
Word: "offerings" Heuristic: 13.923
Word: "<UNK>" Heuristic: 13.923
Word: "inspection" Heuristic: 13.923
Word: "basins" Heuristic: 13.923
Word: "presidents" Heuristic: 13.923
Word: "dollars" Heuristic: 13.923
Word: "experiment" Heuristic: 

Word: "telescope" Heuristic: 13.905
Word: "platforms" Heuristic: 13.905
Word: "coasts" Heuristic: 13.905
Word: "planks" Heuristic: 13.905
Word: "teachers" Heuristic: 13.905
Word: "restaraunt" Heuristic: 13.905
Word: "soldering" Heuristic: 13.905
Word: "butt" Heuristic: 13.905
Word: "graduate" Heuristic: 13.904
Word: "conducting" Heuristic: 13.904
Word: "tracking" Heuristic: 13.904
Word: "littering" Heuristic: 13.904
Word: "roomy" Heuristic: 13.904
Word: "avocados" Heuristic: 13.904
Word: "coffees" Heuristic: 13.904
Word: "cheers" Heuristic: 13.904
Word: "<UNK>" Heuristic: 13.904
Word: "turrets" Heuristic: 13.904
Word: "<UNK>" Heuristic: 13.904
Word: "unison" Heuristic: 13.904
Word: "haze" Heuristic: 13.904
Word: "dr." Heuristic: 13.904
Word: "frosty" Heuristic: 13.904
Word: "saluting" Heuristic: 13.903
Word: "instruction" Heuristic: 13.903
Word: "mammal" Heuristic: 13.903
Word: "croquet" Heuristic: 13.903
Word: "reference" Heuristic: 13.903
Word: "itch" Heuristic: 13.903
Word: "nonchal

Word: "kentucky" Heuristic: 13.781
Word: "processed" Heuristic: 13.781
Word: "authentic" Heuristic: 13.781
Word: "contrasts" Heuristic: 13.781
Word: "cuisine" Heuristic: 13.781
Word: "spam" Heuristic: 13.781
Word: "technical" Heuristic: 13.781
Word: "reserved" Heuristic: 13.780
Word: "continue" Heuristic: 13.780
Word: "it.." Heuristic: 13.780
Word: "clack" Heuristic: 13.780
Word: "drape" Heuristic: 13.780
Word: "kayaks" Heuristic: 13.780
Word: "diorama" Heuristic: 13.780
Word: "<UNK>" Heuristic: 13.780
Word: "bannister" Heuristic: 13.780
Word: "cigar" Heuristic: 13.780
Word: "betting" Heuristic: 13.780
Word: "cellar" Heuristic: 13.780
Word: "blackbird" Heuristic: 13.780
Word: "exhaust" Heuristic: 13.779
Word: "stir-fry" Heuristic: 13.779
Word: "spatulas" Heuristic: 13.779
Word: "chocolate-covered" Heuristic: 13.779
Word: "imposed" Heuristic: 13.779
Word: "fireplug" Heuristic: 13.779
Word: "contestant" Heuristic: 13.779
Word: "cantaloupe" Heuristic: 13.779
Word: "actor" Heuristic: 13.77

Word: "pallet" Heuristic: 13.571
Word: "nad" Heuristic: 13.571
Word: "allot" Heuristic: 13.571
Word: "temperature" Heuristic: 13.571
Word: "bagels" Heuristic: 13.570
Word: "result" Heuristic: 13.570
Word: "lincoln" Heuristic: 13.570
Word: "bp" Heuristic: 13.570
Word: "successfully" Heuristic: 13.570
Word: "fragile" Heuristic: 13.570
Word: "capable" Heuristic: 13.570
Word: "roundabout" Heuristic: 13.570
Word: "bandaged" Heuristic: 13.570
Word: "concealed" Heuristic: 13.570
Word: "patchwork" Heuristic: 13.569
Word: "papered" Heuristic: 13.569
Word: "sell" Heuristic: 13.569
Word: "bellow" Heuristic: 13.569
Word: "acrobatic" Heuristic: 13.569
Word: "trashed" Heuristic: 13.569
Word: "vision" Heuristic: 13.569
Word: "financial" Heuristic: 13.569
Word: "punching" Heuristic: 13.569
Word: "launches" Heuristic: 13.568
Word: "ceremonial" Heuristic: 13.568
Word: "differnt" Heuristic: 13.568
Word: "sips" Heuristic: 13.568
Word: "pedals" Heuristic: 13.568
Word: "congratulating" Heuristic: 13.568
Wor

Word: "underwater" Heuristic: 13.143
Word: "sin" Heuristic: 13.142
Word: "valley" Heuristic: 13.142
Word: "repairing" Heuristic: 13.141
Word: "quaint" Heuristic: 13.141
Word: "loving" Heuristic: 13.140
Word: "husband" Heuristic: 13.139
Word: "cucumbers" Heuristic: 13.139
Word: "wiped" Heuristic: 13.139
Word: "signaling" Heuristic: 13.139
Word: "peson" Heuristic: 13.138
Word: "microwaves" Heuristic: 13.138
Word: "tusked" Heuristic: 13.138
Word: "assisting" Heuristic: 13.138
Word: "grind" Heuristic: 13.138
Word: "nuzzle" Heuristic: 13.137
Word: "wrestling" Heuristic: 13.137
Word: "clutching" Heuristic: 13.137
Word: "closet" Heuristic: 13.137
Word: "ivy" Heuristic: 13.136
Word: "indicate" Heuristic: 13.136
Word: "winning" Heuristic: 13.135
Word: "similar" Heuristic: 13.135
Word: "plethora" Heuristic: 13.134
Word: "clip" Heuristic: 13.134
Word: "company" Heuristic: 13.134
Word: "rubs" Heuristic: 13.134
Word: "stalls" Heuristic: 13.134
Word: "cupboard" Heuristic: 13.133
Word: "sport" Heuris

Word: "electric" Heuristic: 10.974
Word: "church" Heuristic: 10.974
Word: "rocking" Heuristic: 10.972
Word: "caption" Heuristic: 10.972
Word: "pool" Heuristic: 10.971
Word: "short" Heuristic: 10.970
Word: "males" Heuristic: 10.970
Word: "garage" Heuristic: 10.966
Word: "whole" Heuristic: 10.965
Word: "six" Heuristic: 10.962
Word: "alone" Heuristic: 10.960
Word: "petting" Heuristic: 10.960
Word: "clocks" Heuristic: 10.960
Word: "pickup" Heuristic: 10.958
Word: "oranges" Heuristic: 10.953
Word: "doll" Heuristic: 10.950
Word: "flowers" Heuristic: 10.948
Word: "within" Heuristic: 10.947
Word: "wrought" Heuristic: 10.947
Word: "pay" Heuristic: 10.946
Word: "break" Heuristic: 10.946
Word: "wear" Heuristic: 10.946
Word: "points" Heuristic: 10.942
Word: "touching" Heuristic: 10.942
Word: "strapped" Heuristic: 10.940
Word: "fashioned" Heuristic: 10.940
Word: "grinding" Heuristic: 10.940
Word: "when" Heuristic: 10.938
Word: "puts" Heuristic: 10.935
Word: "farmer" Heuristic: 10.932
Word: "safety"